In [ ]:
!pip install -U transformers

In [ ]:
import numpy as np
import pandas as pd 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 
import json
from tqdm import tqdm 
import torch
import re 

In [ ]:
data = pd.read_csv('/kaggle/input/top-level-instructions-ru/ru_samples.csv')
train_data = pd.read_csv('/kaggle/input/top-level-instructions-ru/ru_samples_train.csv')
test_data = pd.read_csv('/kaggle/input/top-level-instructions-ru/ru_samples_val.csv')
print()
print(data.head())

In [ ]:
if torch.cuda.is_available():
    print('Automatic Mixed Precision (AMP) is supported.')
else:
    print('Automatic Mixed Precision (AMP) is not supported.')

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [ ]:
access_token = user_secrets.get_secret("hf_token")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=access_token)

In [ ]:
def generate_responses(instructions, criteria):
    '''
    Функция для генерации ответа с оценкой в формате, подходящем для создания DataFrame
    '''
    results = []
    for instruction in tqdm(instructions):
        prompt = (
            "Отвечай на русском языке. " + instruction + 
            ". Дай краткий и содержательный ответ, затем оцени ответ от 1 до 5 по критериям: " + criteria +
            ". Важно! Ответ должен быть в таком формате: ответ, оценка от 1 до 5 без лишнего текста и символов. Не нужно вставлять объяснения ответа и его качества."
        )
        
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
        outputs = model.generate(input_ids, max_new_tokens=1024)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.replace(prompt, "").strip()

        try:
            answer_start = (response.find("**Твой ответ:**") or response.find("**Ответ:**") or response.find("**Ваш ответ:**"))
    
            if answer_start == -1:
                answer = response.strip()
            else:
                answer_start += len("**Твой ответ:**") if answer_start != -1 else 0
                answer_end = response.find("**Оценка:**")
                answer = response[answer_start:answer_end].strip()

            estimate_start = response.find("**Оценка:**") + len("**Оценка:**")
            estimate = response[estimate_start:].strip()

            results.append({'instruction': instruction, 'answer': answer[10:-13], 'estimate': int(estimate.strip())})
            #print("Ответ:", answer[10:-13])
            #print("Оценка:", estimate)
        except (ValueError, TypeError) as e:
            pass
            #print(f"Ошибка преобразования: {e}. Значение: {estimate} не может быть преобразовано в int.")

    df = pd.DataFrame(results)
    return df

# ". Важно! Ответ должен быть в формате JSON без лишнего текста и не выводи ответ между этих символов ```json```, он должен начинаться с { и заканчиваться }: {\"answer\": \"[твой ответ]\", \"estimate\": [оценка от 1 до 5]}."

In [ ]:
train_data_alignment = train_data['text']
test_data_alignment = test_data['text']

print(train_data_alignment.head())
print(test_data_alignment.head())

**Создаём выборку для трейн**

In [ ]:
criteria = "оценка 5 — идеально краткий, полезный и точный ответ, который экономен в словах, избегает повторений и избыточности; оценка 1 — перегружен ненужной информацией, содержит ошибки, ответ не имеет отношения к вопросу, перегружен ненужными словами"

responses_train = generate_responses(train_data_alignment, criteria)


In [ ]:
responses_train.head(-1)

In [ ]:
responses_train.to_csv('data4alignment_train.csv', index=False)

**Создаём выборку для test**

In [ ]:
responses_test = generate_responses(test_data_alignment, criteria)

In [ ]:
responses_test.head()

In [ ]:
responses_test.to_csv('data4alignment_test.csv', index=False)